# Import Dependencies

In [ ]:
import sys
sys.path.append(r'C:\Users\YH006_new\Desktop\Fingers-Gesture-Recognition')

import numpy as np
import torch
import Source.fgr.models as models
from Source.fgr.utils import train_test_split_by_gesture

from Source.fgr.pipelines import Data_Pipeline
from Source.fgr.data_manager import Data_Manager
from warnings import simplefilter
from pathlib import Path
from importlib import reload

# ignore some warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

# Construct a Data Manager

In [ ]:
# pipeline definition and data manager creation
data_path = Path(r'I:/My Drive/finger gesture recognition/')
pipeline = Data_Pipeline(base_data_files_path=data_path)  # configure the data pipeline you would like to use (check pipelines module for more info)
subject = 19
dm = Data_Manager([subject], pipeline)
print(dm.data_info())

# task 1 - single subject, 80/20 train/test split from position 4 all sessions


## data extraction

In [ ]:
# extract datasets from the data manager - labels format: "<subject>_<session>_<position>_<gesture>_<iteration_number>"
dataset = dm.get_dataset(experiments=[f'{subject:03d}_*_4'], add_exp_name=True)

data = dataset[0]
labels = dataset[1]

# train test split
data_train, data_test, labels_train, labels_test = train_test_split_by_gesture(data, labels=labels, test_size=0.2)

# reshape the data to match the model architecture
data_train = data_train.reshape(data_train.shape[0], 1, 4, 4)  # reshape to fit the CNN input
data_test = data_test.reshape(data_test.shape[0], 1, 4, 4)  # reshape to fit the CNN input

## model training

In [ ]:
# set and train a model (cv or not)
model = models.Net(num_classes=10, dropout_rate=0.1)
model.fit_model(data_train, labels_train, val_data=data_test, val_labels=labels_test, num_epochs=120, batch_size=64,
                lr=0.001, l2_weight=0.0001)

## model evaluation

In [ ]:
model.evaluate_model(model.val_data, model.val_labels, cm_title=f'subject {subject} - task 1')

# task 2 - single subject, 80/20 train/test split from all sessions

## data extraction

In [ ]:
# extract datasets from the data manager - labels format: "<subject>_<session>_<position>_<gesture>_<iteration_number>"
dataset = dm.get_dataset(experiments=[f'{subject:03d}_*_*'], add_exp_name=True)

data = dataset[0]
labels = dataset[1]

# train test split
data_train, data_test, labels_train, labels_test = train_test_split_by_gesture(data, labels=labels, test_size=0.2)

# reshape the data to match the model architecture
data_train = data_train.reshape(data_train.shape[0], 1, 4, 4)  # reshape to fit the CNN input
data_test = data_test.reshape(data_test.shape[0], 1, 4, 4)  # reshape to fit the CNN input

## model training

In [ ]:
# set and train a model (cv or not)
model = models.Net(num_classes=10, dropout_rate=0.1)
model.fit_model(data_train, labels_train, val_data=data_test, val_labels=labels_test, num_epochs=120, batch_size=64,
                lr=0.001, l2_weight=0.0001)

## model evaluation

In [ ]:
model.evaluate_model(model.val_data, model.val_labels, cm_title=f'subject {subject} - task 1')

# task 3 - using 70% of session 2 as testing data (the rest is used for training)

## data extraction

In [ ]:
# extract datasets from the data manager - labels format: "<subject>_<session>_<position>_<gesture>_<iteration_number>"
dataset_1 = dm.get_dataset(experiments=[f'{subject:03d}_1_*'], add_exp_name=True)
dataset_2 = dm.get_dataset(experiments=[f'{subject:03d}_2_*'], add_exp_name=True)

data_1 = dataset_2[0]
labels_1 = dataset_2[1]

data_2 = dataset_1[0]
labels_2 = dataset_1[1]

# train test split
data_2_train, data_2_test, labels_2_train, labels_2_test = train_test_split_by_gesture(data_2, labels=labels_2, test_size=0.7)

# reshape the data to match the model architecture input
data_1 = data_1.reshape(data_1.shape[0], 1, 4, 4)
data_2_train = data_2_train.reshape(data_2_train.shape[0], 1, 4, 4)
data_2_test = data_2_test.reshape(data_2_test.shape[0], 1, 4, 4)

## model training

In [ ]:
# set and train a model (cv or not)
model = models.Net(num_classes=10, dropout_rate=0.1)
# model = torch.compile(model)  # optimize for faster training
model.fit_model(data_1, labels_1, num_epochs=120, batch_size=64, lr=0.001, l2_weight=0.0001)
model.fit_model(data_2_train, labels_2_train, val_data=data_2_test, val_labels=labels_2_test, num_epochs=70,
                batch_size=64, lr=0.001, l2_weight=0.0001)

## model evaluation

In [ ]:
model.evaluate_model(data_2_test, labels_2_test, cm_title=f'subject {subject} - task 2')